In [90]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import linregress

In [91]:
data = pd.read_excel('../data/NO2_NO_NH3_freq_meas_data_2021-02-16.xlsx', sheet_name =  1)

In [107]:
N

27

In [92]:
data.head()

,NO2,NO,NH3,1,2,3,4,5,6,7,...,1285,1286,1287,1288,1289,1290,1291,1292,1293,1294
0,50,100,25,0.000263,0.000263,0.000263,0.000263,0.000262,0.000262,0.000262,...,0.000245,0.000245,0.000245,0.000245,0.000245,0.000245,0.000245,0.000245,0.000245,0.000245
1,100,25,100,0.000244,0.000244,0.000244,0.000244,0.000244,0.000244,0.000244,...,0.000260,0.000260,0.000260,0.000260,0.000260,0.000260,0.000260,0.000260,0.000260,0.000260
2,25,100,50,0.000259,0.000259,0.000259,0.000259,0.000259,0.000258,0.000258,...,0.000248,0.000248,0.000248,0.000248,0.000248,0.000248,0.000248,0.000248,0.000248,0.000247
3,50,25,100,0.000246,0.000246,0.000247,0.000247,0.000247,0.000247,0.000247,...,0.000262,0.000262,0.000262,0.000262,0.000261,0.000261,0.000261,0.000261,0.000260,0.000260
4,100,100,25,0.000257,0.000257,0.000257,0.000257,0.000256,0.000256,0.000256,...,0.000243,0.000243,0.000243,0.000243,0.000243,0.000243,0.000243,0.000243,0.000243,0.000243


In [93]:
# Raw response
response = np.asarray(data.loc[:, list(range(1,n_time+1))])

In [94]:
# Gas concentrations
Y = np.asarray(data[['NO2', 'NO', 'NH3']])

In [95]:
n_time = 1000
n_splits = 25
N = Y.shape[0]

In [96]:
# Time steps
time = np.asarray(data.columns[3:n_time+3])

In [97]:
time_splits = np.array_split(time, n_splits)

In [98]:
response_splits = np.array_split(response, n_splits, axis = 1)

In [132]:
len(time_splits)

25

In [99]:
avg = np.zeros((n, num_splits))

for i, resp in zip(range(n),response_splits):
    # Computing average per split
    avg[:,i] = np.average(resp, axis =1)

In [141]:
slope = np.zeros((N, n_splits))

for split, j in zip(response_splits, range(n_splits)):
    for row, i in zip(split, range(N)):
        slope[i, j] = linregress(list(range(40)), row)[0]

In [152]:
Y.shape

(27, 3)

In [151]:
X = np.concatenate((np.asarray(avg), slope), axis = 1)
    
    
X.shape

(27, 50)

In [153]:
from sklearn.preprocessing import scale 
from sklearn import model_selection
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.cross_decomposition import PLSRegression, PLSSVD
from sklearn.metrics import mean_squared_error
pca = PCA()
X_reduced = pca.fit_transform(scale(X))

In [163]:
regr = LinearRegression()
regr.fit(X_reduced[:,:7], Y)

LinearRegression()

In [158]:
pca.explained_variance_ratio_

array([4.70866194e-01, 2.44829628e-01, 1.39509337e-01, 6.17420420e-02,
       3.58478742e-02, 2.07171182e-02, 1.35629382e-02, 6.66114273e-03,
       2.74885182e-03, 1.24816616e-03, 7.59576894e-04, 4.62055896e-04,
       3.34719719e-04, 2.18265409e-04, 1.27138565e-04, 1.03840313e-04,
       5.57739304e-05, 4.70186122e-05, 4.11812432e-05, 3.22855566e-05,
       2.28688659e-05, 1.89528577e-05, 1.61076663e-05, 1.29286985e-05,
       8.12998246e-06, 5.86303763e-06, 3.42641242e-33])

In [160]:
n = len(X_reduced)
kf_10 = model_selection.KFold( n_splits=10, shuffle=True, random_state=1)

regr = LinearRegression()
mse = []

# Calculate MSE with only the intercept (no principal components in regression)
score = -1*model_selection.cross_val_score(regr, np.ones((n,1)), Y.ravel(), cv=kf_10, scoring='neg_mean_squared_error').mean()    
mse.append(score)

# Calculate MSE using CV for the 19 principle components, adding one component at the time.
for i in np.arange(1, 20):
    score = -1*model_selection.cross_val_score(regr, X_reduced[:,:i], Y.ravel(), cv=kf_10, scoring='neg_mean_squared_error').mean()
    mse.append(score)
    
# Plot results    
plt.plot(mse, '-v')
plt.xlabel('Number of principal components in regression')
plt.ylabel('MSE')
plt.title('Salary')
plt.xlim(xmin=-1);

ValueError: Found input variables with inconsistent numbers of samples: [27, 81]